In [48]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization, Average
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras import applications
from keras.applications import ResNet50
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
import keras_metrics
from sklearn import metrics
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from tqdm import tqdm
import os, sys
from keras.engine.topology import Input
import h5py
from keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import classification_report, confusion_matrix




from keras import optimizers


trainLabelsPath = "traininglabels.csv"
trainImagesPath = "/Users/HS/Downloads/widsdatathon2019/train_images/"

testFolderPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_test_data"
testImagesPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_test_data/"

holdFolderPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_holdout_data"
holdImagesPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_holdout_data/"

subFilePath = "finalSubmission_underSamp_ResNet50_TrainAll.csv"

In [ ]:

trainLabelsPath = "/home/ubuntu/WIDS/traininglabels.csv"
trainImagesPath = "/home/ubuntu/WIDS/train_images/"

testFolderPath = "/home/ubuntu/WIDS/leaderboard_test_data"
testImagesPath = "/home/ubuntu/WIDS/leaderboard_test_data/"

holdFolderPath = "/home/ubuntu/WIDS/leaderboard_holdout_data"
holdImagesPath = "/home/ubuntu/WIDS/leaderboard_holdout_data/"

subFilePath = "finalSubmission_ResNet50_CV_ConfusionMatrix.csv"

# Upload training data file

In [50]:
df = pd.read_csv(trainLabelsPath)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15244 entries, 0 to 15243
Data columns (total 3 columns):
image_id       15244 non-null object
has_oilpalm    15244 non-null int64
score          15244 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 357.4+ KB


In [52]:
df[df.has_oilpalm == 1].count() # only 6% of the images have palm oil plantations

image_id       942
has_oilpalm    942
score          942
dtype: int64

In [53]:
df.score.sort_values()[:5]
# some images have very small scores. Perhaps use a threshold like 0.5 and give 0.5 or less, label 0

1062    0.3887
72      0.3944
353     0.3954
1404    0.3963
270     0.3965
Name: score, dtype: float64

In [54]:
df = df.sort_values(by='image_id').reset_index(drop=True)

In [55]:
df_hasPalm = df[(df.has_oilpalm == 1)]
df_noPalm = df[(df.has_oilpalm == 0) & (df.score == 1)]

In [56]:
df_noPalm_samp = df_noPalm.sample(1000, random_state=42)
df_noPalm_samp.shape

(1000, 3)

In [57]:
df_final = df_hasPalm.append(df_noPalm_samp).sort_values(by='image_id').reset_index(drop=True)

In [58]:
df_final.head()

,image_id,has_oilpalm,score
0,img_000152018.jpg,0,1.0
1,img_000192018.jpg,0,1.0
2,img_000282018.jpg,0,1.0
3,img_000312017.jpg,0,1.0
4,img_000342017.jpg,0,1.0


In [59]:
type(df_final.image_id)

pandas.core.series.Series

In [60]:
img_height=256
img_width=256

In [61]:
# Reading images from a folder and converting to numpy array
train_image = []
for i in tqdm(range(len(df_final))):
    img = image.load_img(trainImagesPath + df_final.image_id[i], target_size=(img_height,img_width))
    img = image.img_to_array(img)
    img = img/255
    
    
    train_image.append(img)
X = np.array(train_image)
X

100%|██████████| 1942/1942 [00:06<00:00, 306.71it/s]


array([[[[ 0.05098039,  0.11764706,  0.05490196],
         [ 0.03529412,  0.10196079,  0.03921569],
         [ 0.01960784,  0.08627451,  0.02352941],
         ..., 
         [ 0.02745098,  0.08235294,  0.02352941],
         [ 0.03921569,  0.10196079,  0.05098039],
         [ 0.08235294,  0.14509805,  0.09411765]],

        [[ 0.00392157,  0.07058824,  0.00784314],
         [ 0.        ,  0.0627451 ,  0.        ],
         [ 0.01568628,  0.07058824,  0.00392157],
         ..., 
         [ 0.02352941,  0.09019608,  0.02745098],
         [ 0.03137255,  0.09803922,  0.03529412],
         [ 0.0627451 ,  0.1254902 ,  0.07450981]],

        [[ 0.00784314,  0.0627451 ,  0.        ],
         [ 0.        ,  0.05490196,  0.        ],
         [ 0.01960784,  0.07450981,  0.00784314],
         ..., 
         [ 0.03137255,  0.09803922,  0.02745098],
         [ 0.03921569,  0.10588235,  0.04313726],
         [ 0.05490196,  0.12156863,  0.05882353]],

        ..., 
        [[ 0.07058824,  0.14509805,

In [62]:
y = df_final['has_oilpalm']
#y = to_categorical(y)
y

0       0
1       0
2       0
3       0
4       0
5       0
6       1
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      1
18      0
19      0
20      1
21      0
22      1
23      1
24      0
25      0
26      1
27      0
28      1
29      0
       ..
1912    1
1913    0
1914    1
1915    1
1916    1
1917    1
1918    1
1919    0
1920    0
1921    0
1922    1
1923    1
1924    1
1925    0
1926    0
1927    0
1928    0
1929    0
1930    0
1931    0
1932    0
1933    0
1934    0
1935    1
1936    0
1937    1
1938    0
1939    1
1940    0
1941    0
Name: has_oilpalm, Length: 1942, dtype: int64

# Leaderboard Test Data Prediction

In [63]:
testpath = testFolderPath
testimagesList = sorted(os.listdir(testpath))
testimagesList

['img_000022018.jpg',
 'img_000042017.jpg',
 'img_000052017.jpg',
 'img_000062017.jpg',
 'img_000062018.jpg',
 'img_000122017.jpg',
 'img_000142018.jpg',
 'img_000162018.jpg',
 'img_000182017.jpg',
 'img_000262017.jpg',
 'img_000292017.jpg',
 'img_000302018.jpg',
 'img_000342018.jpg',
 'img_000382017.jpg',
 'img_000382018.jpg',
 'img_000422017.jpg',
 'img_000422018.jpg',
 'img_000432017.jpg',
 'img_000472018.jpg',
 'img_000532017.jpg',
 'img_000542018.jpg',
 'img_000552017.jpg',
 'img_000592017.jpg',
 'img_000602018.jpg',
 'img_000622017.jpg',
 'img_000622018.jpg',
 'img_000632018.jpg',
 'img_000672018.jpg',
 'img_000682018.jpg',
 'img_000692017.jpg',
 'img_000702017.jpg',
 'img_000752017.jpg',
 'img_000762017.jpg',
 'img_000812018.jpg',
 'img_000822018.jpg',
 'img_000832018.jpg',
 'img_000842018.jpg',
 'img_000852018.jpg',
 'img_000872018.jpg',
 'img_000882017.jpg',
 'img_000882018.jpg',
 'img_000902018.jpg',
 'img_000912017.jpg',
 'img_000912018.jpg',
 'img_000922018.jpg',
 'img_0009

In [64]:
leaderboard_test_pred = pd.DataFrame(testimagesList , columns=['image_id'])
leaderboard_test_pred

,image_id
0,img_000022018.jpg
1,img_000042017.jpg
2,img_000052017.jpg
3,img_000062017.jpg
4,img_000062018.jpg
5,img_000122017.jpg
6,img_000142018.jpg
7,img_000162018.jpg
8,img_000182017.jpg
9,img_000262017.jpg


In [65]:
test_image = []
for i in tqdm(range(len(testimagesList))):
    img = image.load_img(testImagesPath + testimagesList[i], target_size=(img_height,img_width))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)
test

100%|██████████| 4356/4356 [00:13<00:00, 330.23it/s]


array([[[[ 0.43529412,  0.35294119,  0.23137255],
         [ 0.48235294,  0.40000001,  0.28627452],
         [ 0.52941179,  0.44705883,  0.33333334],
         ..., 
         [ 0.18039216,  0.18431373,  0.12941177],
         [ 0.23137255,  0.23529412,  0.18039216],
         [ 0.30980393,  0.3137255 ,  0.25882354]],

        [[ 0.50196081,  0.41960785,  0.30588236],
         [ 0.52156866,  0.43921569,  0.32549021],
         [ 0.54509807,  0.4627451 ,  0.34901962],
         ..., 
         [ 0.17254902,  0.17647059,  0.12156863],
         [ 0.20392157,  0.20784314,  0.15294118],
         [ 0.27843139,  0.28235295,  0.22745098]],

        [[ 0.61960787,  0.53725493,  0.42352942],
         [ 0.60784316,  0.52549022,  0.41176471],
         [ 0.58823532,  0.50588238,  0.39215687],
         ..., 
         [ 0.18431373,  0.1882353 ,  0.13333334],
         [ 0.19215687,  0.19607843,  0.14117648],
         [ 0.21960784,  0.22352941,  0.16862746]],

        ..., 
        [[ 0.63137257,  0.56470591,

# Leaderboard Holdback Data Prediction

In [66]:
holdpath = holdFolderPath
holdimagesList = sorted(os.listdir(holdpath))
holdimagesList

['img_000012018.jpg',
 'img_000032017.jpg',
 'img_000132018.jpg',
 'img_000162017.jpg',
 'img_000222017.jpg',
 'img_000222018.jpg',
 'img_000252018.jpg',
 'img_000262018.jpg',
 'img_000432018.jpg',
 'img_000482017.jpg',
 'img_000482018.jpg',
 'img_000502017.jpg',
 'img_000572017.jpg',
 'img_000612018.jpg',
 'img_000642017.jpg',
 'img_000642018.jpg',
 'img_000652018.jpg',
 'img_000682017.jpg',
 'img_000722017.jpg',
 'img_000742018.jpg',
 'img_000772018.jpg',
 'img_000842017.jpg',
 'img_000922017.jpg',
 'img_001022017.jpg',
 'img_001092018.jpg',
 'img_001102018.jpg',
 'img_001162017.jpg',
 'img_001172017.jpg',
 'img_001242018.jpg',
 'img_001282017.jpg',
 'img_001282018.jpg',
 'img_001352017.jpg',
 'img_001502017.jpg',
 'img_001612017.jpg',
 'img_001622018.jpg',
 'img_001672018.jpg',
 'img_001822018.jpg',
 'img_001832017.jpg',
 'img_001882017.jpg',
 'img_001962018.jpg',
 'img_002062017.jpg',
 'img_002112018.jpg',
 'img_002182017.jpg',
 'img_002202018.jpg',
 'img_002242017.jpg',
 'img_0023

In [67]:
leaderboard_hold_pred = pd.DataFrame(holdimagesList , columns=['image_id'])
leaderboard_hold_pred

,image_id
0,img_000012018.jpg
1,img_000032017.jpg
2,img_000132018.jpg
3,img_000162017.jpg
4,img_000222017.jpg
5,img_000222018.jpg
6,img_000252018.jpg
7,img_000262018.jpg
8,img_000432018.jpg
9,img_000482017.jpg


In [68]:
hold_image = []
for i in tqdm(range(len(holdimagesList))):
    img = image.load_img(holdImagesPath + holdimagesList[i], target_size=(img_height,img_width))
    img = image.img_to_array(img)
    img = img/255
    hold_image.append(img)
hold = np.array(hold_image)
hold

100%|██████████| 2178/2178 [00:07<00:00, 279.04it/s]


array([[[[ 0.14901961,  0.21176471,  0.09803922],
         [ 0.16078432,  0.22352941,  0.10980392],
         [ 0.15294118,  0.21568628,  0.10196079],
         ..., 
         [ 0.03921569,  0.11764706,  0.00784314],
         [ 0.08235294,  0.16078432,  0.05098039],
         [ 0.11764706,  0.19607843,  0.08627451]],

        [[ 0.14117648,  0.20392157,  0.09019608],
         [ 0.14509805,  0.20784314,  0.09411765],
         [ 0.14509805,  0.20784314,  0.09411765],
         ..., 
         [ 0.03529412,  0.11372549,  0.00392157],
         [ 0.0627451 ,  0.14117648,  0.03137255],
         [ 0.08235294,  0.16078432,  0.05098039]],

        [[ 0.12941177,  0.19215687,  0.07843138],
         [ 0.13725491,  0.2       ,  0.08627451],
         [ 0.14117648,  0.20392157,  0.09019608],
         ..., 
         [ 0.07058824,  0.14509805,  0.03529412],
         [ 0.07058824,  0.14901961,  0.03921569],
         [ 0.0627451 ,  0.14117648,  0.03137255]],

        ..., 
        [[ 0.15686275,  0.21568628,

In [69]:
test_final = test_image+hold_image
test_final = np.array(test_final)

In [70]:
test_set = leaderboard_test_pred.append(leaderboard_hold_pred)

In [75]:
img_height = 256
img_width = 256
epochs = 25

def resnet50(model_input):   
    #ReNet50 model definition
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)
    last = base_model.output
    x = BatchNormalization()(last)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    #x = Dropout(0.5)(x)


    #Freeze
    for layer in base_model.layers:
        layer.trainable = False

    preds = Dense(1, activation='sigmoid')(x)
    model = Model(base_model.input, preds)
    
    #Compile & fit
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', 
                      metrics = [keras_metrics.precision(), keras_metrics.recall(),'accuracy'])
    
    model.fit(X, y, epochs=epochs, verbose=1)
    
    #Unfreeze & fit
    for layer in model.layers:
        layer.trainable = True
    
    #model.fit(X, y, epochs=epochs, verbose=1)
    return model

def InceptionV3(model_input):
    base_model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_tensor=model_input)
    last = base_model.output
    x = BatchNormalization()(last)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    #x = Dropout(0.5)(x)
    
    #Freeze
    for layer in base_model.layers:
        layer.trainable = False
   
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(base_model.input, preds)
    
    #Compile & fit
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', 
                      metrics = [keras_metrics.precision(), keras_metrics.recall(),'accuracy'])
    
    model.fit(X, y, epochs=epochs, verbose=1)
    
    #Unfreeze & fit
    for layer in model.layers:
        layer.trainable = True
    
    #model.fit(X, y, epochs=epochs, verbose=1)
    return model

model_input = Input(shape=(256, 256, 3))
resnet50_model = resnet50(model_input)
InceptionV3_model = InceptionV3(model_input)

ensembled_models = [resnet50_model,InceptionV3_model]
def ensemble(models,model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model1 = Model(model_input,y,name='ensemble')
    return model1


ensemble_model = ensemble(ensembled_models,model_input)
#ensemble_model.summary()

In [ ]:
def resnet50(model_input):   
    #ReNet50 model definition
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)
    last = base_model.output
    x = BatchNormalization()(last)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(base_model.input, preds)
    return model

model_input = Input(shape=(256, 256, 3))
resnet_model = resnet50(model_input)

In [44]:
model = resnet_model

In [47]:
def train_model(model, batch_size, epochs, img_size, x, y, test, n_fold, kf):
    roc_auc = metrics.roc_auc_score
    preds_train = np.zeros(len(x), dtype = np.float)
    preds_test = np.zeros(len(test), dtype = np.float)
    train_scores = []; valid_scores = []

    i = 1

    for train_index, test_index in kf.split(x):
        #print(train_index, ' ',test_index)
        x_train = x[train_index]; x_valid = x[test_index]
        y_train = y[train_index]; y_valid = y[test_index]


        callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, cooldown=1, 
                               verbose=1, min_lr=1e-7),
             ModelCheckpoint(filepath='inception.fold_' + str(i) + '.hdf5', verbose=1,
                             save_best_only=True, save_weights_only=True, mode='auto')]

        train_steps = len(x_train) / batch_size
        valid_steps = len(x_valid) / batch_size
        test_steps = len(test) / batch_size
        
        model = resnet_model

        model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', 
                      metrics = [keras_metrics.precision(), keras_metrics.recall(),'accuracy'])

        model.fit(x_train, y_train, epochs=epochs, verbose=1,
                             callbacks=callbacks, batch_size=16,
                             validation_data=(x_valid, y_valid))

        model.load_weights(filepath='inception.fold_' + str(i) + '.hdf5')

        print('Running validation predictions on fold {}'.format(i))
        preds_valid = model.predict(x_valid,
                                       verbose=1)[:, 0]

        print('Running train predictions on fold {}'.format(i))
        preds_train = model.predict(x_train,
                                       verbose=1)[:, 0]

        valid_score = roc_auc(y_valid, preds_valid)
        train_score = roc_auc(y_train, preds_train)
        print('Val Score:{} for fold {}'.format(valid_score, i))
        print('Train Score: {} for fold {}'.format(train_score, i))

        valid_scores.append(valid_score)
        train_scores.append(train_score)
        print('Avg Train Score:{0:0.5f}, Avg Val Score:{1:0.5f} after {2:0.5f} folds'.format
              (np.mean(train_scores), np.mean(valid_scores), i))

        #print('Running train Confusion Matrix with fold {}'.format(i))
        #train_y_class = preds_train.argmax(axis=-1)
        
        #print("\n Classification Report \n"+ classification_report(y_train, train_y_class))

        #print("\n Confusion Matrix \n")
        #print(confusion_matrix(y_train,train_y_class))
        
        
        #print('Running valid Confusion Matrix with fold {}'.format(i))
        #valid_y_class = preds_valid.argmax(axis=-1)
    
        #print("\n Classification Report \n"+ classification_report(y_valid, valid_y_class))

        #print("\n Confusion Matrix \n")
        #print(confusion_matrix(y_valid,valid_y_class))
    
    
        print('Running test predictions with fold {}'.format(i))

        preds_test_fold = model.predict(test_final,
                                               verbose=1)[:, -1]

        preds_test += preds_test_fold

        print('\n\n')

        i += 1

        if i <= n_fold:
            print('Now beginning training for fold {}\n\n'.format(i))
        else:
            print('Finished training!')

    preds_test /= n_fold


    return preds_test

In [46]:
batch_size = 5
n_fold = 4
img_size = (img_height, img_width)
kf = KFold(n_splits=n_fold, shuffle=True)

test_pred = train_model(model, batch_size, epochs, img_size, X, y, test_final, n_fold, kf)

test_set['has_oilpalm'] = test_pred
test_set.to_csv(subFilePath, index = None)

Train on 1553 samples, validate on 389 samples
Epoch 1/25
  32/1553 [..............................] - ETA: 43:49 - loss: 0.0419 - precision: 1.0000 - recall: 1.0000 - acc: 1.0000

KeyboardInterrupt: 